Exercicis per practicar amb regressions.
- Exercici 1
Crea almenys dos models de regressió diferents per intentar predir el millor possible el preu de les vivendes (MEDV) de l'arxiu adjunt.


Decision Tree Regression 

In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as scipy
from scipy import stats
from scipy.stats import ttest_ind
%matplotlib inline

In [66]:
from sklearn import datasets, metrics 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [67]:
df = pd.read_excel('Housing data.xslx.xlsx')
df

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,632,1800,2310,0,5380,65750,6520,40900,1,2960,1530,39690,498,2400
1,2731,0,7070,0,4690,64210,7890,49671,2,2420,1780,39690,914,2160
2,2729,0,7070,0,4690,71850,6110,49671,2,2420,1780,39283,403,3470
3,3237,0,2180,0,4580,69980,4580,60622,3,2220,1870,39463,294,3340
4,6905,0,2180,0,4580,71470,5420,60622,3,2220,1870,39690,533,3620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,6263,0,11930,0,5730,65930,6910,24786,1,2730,2100,39199,967,2240
502,4527,0,11930,0,5730,61200,7670,22875,1,2730,2100,39690,908,2060
503,6076,0,11930,0,5730,69760,9100,21675,1,2730,2100,39690,564,2390
504,10959,0,11930,0,5730,67940,8930,23889,1,2730,2100,39345,648,2200


In [68]:
df.columns

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT', 'MEDV'],
      dtype='object')

In [69]:
df.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
count,5.060000e+02,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524e+05,1136.363636,11136.778656,0.069170,5546.950593,62846.343874,6857.490119,37950.426877,9.549407,4082.371542,1845.553360,35667.403162,1265.306324,2253.280632
std,8.601545e+05,2332.245299,6860.352941,0.253994,1158.776757,7026.171434,2814.886141,21057.101266,8.707259,1685.371161,216.494552,9129.486438,714.106151,919.710409
min,6.320000e+02,0.000000,460.000000,0.000000,3850.000000,35610.000000,290.000000,11296.000000,1.000000,1870.000000,1260.000000,32.000000,173.000000,500.000000
25%,8.204500e+03,0.000000,5190.000000,0.000000,4490.000000,58855.000000,4502.500000,21001.750000,4.000000,2790.000000,1740.000000,37537.750000,695.000000,1702.500000
50%,2.565100e+04,0.000000,9690.000000,0.000000,5380.000000,62085.000000,7750.000000,32074.500000,5.000000,3300.000000,1905.000000,39144.000000,1136.000000,2120.000000
75%,3.677082e+05,1250.000000,18100.000000,0.000000,6240.000000,66235.000000,9407.500000,51884.250000,24.000000,6660.000000,2020.000000,39622.500000,1695.500000,2500.000000
max,8.897620e+06,10000.000000,27740.000000,1.000000,8710.000000,87800.000000,10000.000000,121265.000000,24.000000,7110.000000,2200.000000,39690.000000,3797.000000,5000.000000


Data Preparation

In [70]:
df.columns = ['Crime Rate', 'Residential Zone', 'Non Retail bsn', 'Dummy Var', 'Nitric Ox Concetration', 'Room Average', 'Owners', 'Dist Boston Jobs', 'Index Highways Distance', 'Tax rate', 'School Ratio', 'Blacks Average', 'Perc Status Popul', 'Median Value Homes']
df.columns

Index(['Crime Rate', 'Residential Zone', 'Non Retail bsn', 'Dummy Var',
       'Nitric Ox Concetration', 'Room Average', 'Owners', 'Dist Boston Jobs',
       'Index Highways Distance', 'Tax rate', 'School Ratio', 'Blacks Average',
       'Perc Status Popul', 'Median Value Homes'],
      dtype='object')

In [71]:
df

,Crime Rate,Residential Zone,Non Retail bsn,Dummy Var,Nitric Ox Concetration,Room Average,Owners,Dist Boston Jobs,Index Highways Distance,Tax rate,School Ratio,Blacks Average,Perc Status Popul,Median Value Homes
0,632,1800,2310,0,5380,65750,6520,40900,1,2960,1530,39690,498,2400
1,2731,0,7070,0,4690,64210,7890,49671,2,2420,1780,39690,914,2160
2,2729,0,7070,0,4690,71850,6110,49671,2,2420,1780,39283,403,3470
3,3237,0,2180,0,4580,69980,4580,60622,3,2220,1870,39463,294,3340
4,6905,0,2180,0,4580,71470,5420,60622,3,2220,1870,39690,533,3620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,6263,0,11930,0,5730,65930,6910,24786,1,2730,2100,39199,967,2240
502,4527,0,11930,0,5730,61200,7670,22875,1,2730,2100,39690,908,2060
503,6076,0,11930,0,5730,69760,9100,21675,1,2730,2100,39690,564,2390
504,10959,0,11930,0,5730,67940,8930,23889,1,2730,2100,39345,648,2200


In [72]:
df.isnull().sum()

Crime Rate                 0
Residential Zone           0
Non Retail bsn             0
Dummy Var                  0
Nitric Ox Concetration     0
Room Average               0
Owners                     0
Dist Boston Jobs           0
Index Highways Distance    0
Tax rate                   0
School Ratio               0
Blacks Average             0
Perc Status Popul          0
Median Value Homes         0
dtype: int64

In [73]:
df.dropna()

,Crime Rate,Residential Zone,Non Retail bsn,Dummy Var,Nitric Ox Concetration,Room Average,Owners,Dist Boston Jobs,Index Highways Distance,Tax rate,School Ratio,Blacks Average,Perc Status Popul,Median Value Homes
0,632,1800,2310,0,5380,65750,6520,40900,1,2960,1530,39690,498,2400
1,2731,0,7070,0,4690,64210,7890,49671,2,2420,1780,39690,914,2160
2,2729,0,7070,0,4690,71850,6110,49671,2,2420,1780,39283,403,3470
3,3237,0,2180,0,4580,69980,4580,60622,3,2220,1870,39463,294,3340
4,6905,0,2180,0,4580,71470,5420,60622,3,2220,1870,39690,533,3620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,6263,0,11930,0,5730,65930,6910,24786,1,2730,2100,39199,967,2240
502,4527,0,11930,0,5730,61200,7670,22875,1,2730,2100,39690,908,2060
503,6076,0,11930,0,5730,69760,9100,21675,1,2730,2100,39690,564,2390
504,10959,0,11930,0,5730,67940,8930,23889,1,2730,2100,39345,648,2200


- Train Test Split 

In [74]:
# Use numpy to convert to arrays
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split 

# All the features except the Target or Label Column which is 'Median Value Homes' 
Features_Variables_Independientes_X = df.drop('Median Value Homes', axis = 1)

# Labels are the value we want to predict:'Median Value Homes'
Labels_Variable_Dependiente_y = df[['Median Value Homes']]


X_train, X_test, y_train, y_test = train_test_split(Features_Variables_Independientes_X,Labels_Variable_Dependiente_y, 
 random_state=0)



In [75]:
print('Training Features Shape / Variables_Independientes_X:', X_train.shape)
print('Training Labels Shape / Variable_Dependiente_y :', y_train.shape)
print('Testing Features Shape / Variables_Independientes_X:', X_test.shape)
print('Testing Labels Shape / Variable_Dependiente_y:', y_test.shape)

Training Features Shape / Variables_Independientes_X: (379, 13)
Training Labels Shape / Variable_Dependiente_y : (379, 1)
Testing Features Shape / Variables_Independientes_X: (127, 13)
Testing Labels Shape / Variable_Dependiente_y: (127, 1)


- Model Random Forest Regression 

In [76]:
regressor = RandomForestRegressor(n_estimators = 100, random_state = 42)

In [77]:
regressor.fit(X_train, y_train)

C:\Users\flore\AppData\Local\Temp\ipykernel_10308\938670134.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regressor.fit(X_train, y_train)


RandomForestRegressor(random_state=42)

In [84]:
y_prediction = regressor.predict(X_test)

In [85]:
y_prediction

array([2348.1, 2827.5, 2231.7, 1085.5, 2037.8, 2076.3, 2099.6, 2011.4,
       2091.1, 1882.2,  831.8, 1459.8, 1458.7,  833.8, 4810.4, 3393.3,
       2125.9, 3500.1, 2515.7, 2135.3, 2373.5, 2205.9, 1941.9, 2436.2,
       2018.8, 1740.9, 1847.6, 1593.9, 4315.7, 1851.4, 1436. , 1727.3,
       1978. , 2093.8, 2300.7, 1717.3,  812.2, 2795.7, 1375.5, 1473.2,
       2270.4, 2069.4, 2205.9, 1505.2, 2292.3, 2263.9, 2071.6, 1679.2,
       1451.6, 2501.1, 1670.2, 1875.1, 2195.9, 4079.5, 1522. , 1961.2,
       1938.6, 1890.8, 2100.3, 2052.6, 2139.8, 2104.3, 3330.9, 2720. ,
       1770. , 2598.5, 1627.1, 1787.2, 1663.6, 2223.4, 2016.7, 2276.8,
       2442.8, 3178.6, 2898.2,  825.9, 4244. , 2202.7, 2280.3, 2024. ,
       2709.2, 1796.9, 2106. , 4221.9, 4108.7, 2431.4, 2307.7, 1434.1,
       2667.6, 1583.2, 1903. , 1208.7, 2232.3, 3078.5, 2139. , 2157.6,
       1228.4, 2371.5, 1494.3, 1907. , 2393.6, 1985.3, 2812. , 2119.7,
       2779.3, 2054.1,  830.3, 1921.4, 2168.9, 2310.4, 3315.3, 1312.5,
      

In [90]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(X_train, y_train);

C:\Users\flore\AppData\Local\Temp\ipykernel_10308\2984729332.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train);


In [116]:
# Use the forest's predict method on the test data
predictions = rf.predict(X_test)
# Calculate the absolute errors
errors = abs(predictions(y_prediction)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:' round(np.mean(errors), 2), 'degrees.')

SyntaxError: invalid syntax (1187999963.py, line 6)

- Exercici 2
Compara’ls en base al MSE i al R2.


In [88]:
#Mean Squared Error
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_prediction)

# (el y_test es el mismo que X_train, X_test, y_train, y_test = train_test_split(Features_Variables_Independientes_X,Labels_Variable_Dependiente_y, 
 #random_state=0))



175713.23527559056

R2

El Model Random Forest Regression es en comparació amb el model Linear Regression, un millor model per predecir la variable objectiu: "Median Value Homes"; malgrat dels canvis dels paràmetres (Features variables o Variables dependents), el model Random Forest Regression es manté amb un R2 Score més alt que el del model Lineal Regression.

In [89]:
#R² Score
from sklearn.metrics import r2_score
r2_score(y_test, y_prediction)

0.7849261287501439

Exercici 3 Entrena’ls utilitzant els diferents paràmetres que admeten per intentar millorar-ne la predicció

Per veure el impacte que poden tenir altres variables, he seleccionat aquestes variables independients: 'Owners','Perc Status Popul', 'Crime Rate', 'Nitric Ox Concetration', 'Dist Boston Jobs'

In [117]:
df

,Crime Rate,Residential Zone,Non Retail bsn,Dummy Var,Nitric Ox Concetration,Room Average,Owners,Dist Boston Jobs,Index Highways Distance,Tax rate,School Ratio,Blacks Average,Perc Status Popul,Median Value Homes
0,632,1800,2310,0,5380,65750,6520,40900,1,2960,1530,39690,498,2400
1,2731,0,7070,0,4690,64210,7890,49671,2,2420,1780,39690,914,2160
2,2729,0,7070,0,4690,71850,6110,49671,2,2420,1780,39283,403,3470
3,3237,0,2180,0,4580,69980,4580,60622,3,2220,1870,39463,294,3340
4,6905,0,2180,0,4580,71470,5420,60622,3,2220,1870,39690,533,3620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,6263,0,11930,0,5730,65930,6910,24786,1,2730,2100,39199,967,2240
502,4527,0,11930,0,5730,61200,7670,22875,1,2730,2100,39690,908,2060
503,6076,0,11930,0,5730,69760,9100,21675,1,2730,2100,39690,564,2390
504,10959,0,11930,0,5730,67940,8930,23889,1,2730,2100,39345,648,2200


In [118]:
# Use numpy to convert to arrays
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split 

# All the features except the Target or Label Column which is 'Median Value Homes' 
Features_Variables_Independientes_X = dfFeatures_Variables_Independientes_X = df[['Owners','Perc Status Popul', 'Crime Rate', 'Nitric Ox Concetration', 'Dist Boston Jobs']]

# Labels are the value we want to predict:'Median Value Homes'
Labels_Variable_Dependiente_y = df[['Median Value Homes']]


X_train, X_test, y_train, y_test = train_test_split(Features_Variables_Independientes_X,Labels_Variable_Dependiente_y, 
 random_state=0)

In [119]:
print('Training Features Shape / Variables_Independientes_X:', X_train.shape)
print('Training Labels Shape / Variable_Dependiente_y :', y_train.shape)
print('Testing Features Shape / Variables_Independientes_X:', X_test.shape)
print('Testing Labels Shape / Variable_Dependiente_y:', y_test.shape)

Training Features Shape / Variables_Independientes_X: (379, 5)
Training Labels Shape / Variable_Dependiente_y : (379, 1)
Testing Features Shape / Variables_Independientes_X: (127, 5)
Testing Labels Shape / Variable_Dependiente_y: (127, 1)


In [120]:
regressor = RandomForestRegressor(n_estimators = 100, random_state = 42)

In [121]:
regressor.fit(X_train, y_train)

C:\Users\flore\AppData\Local\Temp\ipykernel_10308\938670134.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regressor.fit(X_train, y_train)


RandomForestRegressor(random_state=42)

In [122]:
y_prediction = regressor.predict(X_test)

In [123]:
y_prediction

array([2444.5, 2785.6, 2117.8, 1053.4, 2097.4, 2158.4, 2258.5, 2136.8,
       2217.2, 2028.1,  868.9, 1446.2, 1479.9,  803.7, 4934. , 2872.6,
       2190.7, 3307.4, 2969.5, 2396.3, 2504.7, 2112.4, 2116.5, 2800.6,
       2092.6, 1807.1, 1866.4, 1641.9, 3652.7, 1810.4, 1390.7, 1862.4,
       1996.2, 1970.4, 2828.7, 1838.2,  802.6, 2837.4, 1407.4, 1489.9,
       2200.1, 2013.1, 2131.4, 1491.8, 2258.5, 2256.7, 1949.7, 1639.2,
       1423.9, 2685.8, 1777.2, 1963. , 2207.9, 2903.2, 1618.5, 2049.1,
       2044.4, 2000.7, 2060.1, 1968.4, 2158.1, 2027.5, 3017. , 4326.5,
       2024.5, 3327. , 1599.8, 1801. , 1464.9, 2161.9, 1957.3, 2206.5,
       3019. , 3246.5, 2550.7,  830.1, 3747.7, 2103.5, 2491.4, 2087.4,
       2523.5, 1848.1, 1823.4, 4091.6, 3723.6, 2510. , 2204.7, 1510.8,
       2966.5, 1519.8, 2080.5, 1229.8, 2750.8, 3013.8, 2772.6, 2306. ,
       1236.1, 2634.7, 1535.6, 1875.6, 2382.8, 2027. , 2525.7, 2175.2,
       2019.3, 1961. ,  804.3, 1966.9, 2759.7, 2177.2, 3822.3, 1335.7,
      

In [124]:
#Mean Squared Error
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_prediction)

242024.8857480315

In [125]:
#R² Score
from sklearn.metrics import r2_score
r2_score(y_test, y_prediction)

0.7037603397661398

- Exercici 5
No facis servir la variable del nombre d'habitacions (RM) a l’hora de fer prediccions.


In [132]:
df

,Crime Rate,Residential Zone,Non Retail bsn,Dummy Var,Nitric Ox Concetration,Room Average,Owners,Dist Boston Jobs,Index Highways Distance,Tax rate,School Ratio,Blacks Average,Perc Status Popul,Median Value Homes
0,632,1800,2310,0,5380,65750,6520,40900,1,2960,1530,39690,498,2400
1,2731,0,7070,0,4690,64210,7890,49671,2,2420,1780,39690,914,2160
2,2729,0,7070,0,4690,71850,6110,49671,2,2420,1780,39283,403,3470
3,3237,0,2180,0,4580,69980,4580,60622,3,2220,1870,39463,294,3340
4,6905,0,2180,0,4580,71470,5420,60622,3,2220,1870,39690,533,3620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,6263,0,11930,0,5730,65930,6910,24786,1,2730,2100,39199,967,2240
502,4527,0,11930,0,5730,61200,7670,22875,1,2730,2100,39690,908,2060
503,6076,0,11930,0,5730,69760,9100,21675,1,2730,2100,39690,564,2390
504,10959,0,11930,0,5730,67940,8930,23889,1,2730,2100,39345,648,2200


In [133]:
# Use numpy to convert to arrays
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split 

# All the features except the Target or Label Column which is 'Median Value Homes' 
Features_Variables_Independientes_X = df.drop(columns=['Median Value Homes','Room Average'])

# Labels are the value we want to predict:'Median Value Homes'
Labels_Variable_Dependiente_y = df[['Median Value Homes']]


X_train, X_test, y_train, y_test = train_test_split(Features_Variables_Independientes_X,Labels_Variable_Dependiente_y, 
 random_state=0)

In [134]:
print('Training Features Shape / Variables_Independientes_X:', X_train.shape)
print('Training Labels Shape / Variable_Dependiente_y :', y_train.shape)
print('Testing Features Shape / Variables_Independientes_X:', X_test.shape)
print('Testing Labels Shape / Variable_Dependiente_y:', y_test.shape)

Training Features Shape / Variables_Independientes_X: (379, 12)
Training Labels Shape / Variable_Dependiente_y : (379, 1)
Testing Features Shape / Variables_Independientes_X: (127, 12)
Testing Labels Shape / Variable_Dependiente_y: (127, 1)


In [135]:
regressor = RandomForestRegressor(n_estimators = 100, random_state = 42)

In [136]:
regressor.fit(X_train, y_train)

C:\Users\flore\AppData\Local\Temp\ipykernel_10308\938670134.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regressor.fit(X_train, y_train)


RandomForestRegressor(random_state=42)

In [137]:
y_prediction = regressor.predict(X_test)

In [140]:
y_prediction 

array([2742.8, 2240. , 2376.8, 1088.7, 1975.7, 2130.6, 2239.1, 2032.6,
       2183.2, 1865.1,  832.9, 1479.9, 1446.4,  792.3, 4837.1, 2945.6,
       2204.4, 3777.7, 2816.2, 2149.8, 2341.6, 2194.8, 1950.2, 2705.5,
       2171.3, 1643.9, 1891.5, 1646.2, 3644.2, 1839.7, 1478.6, 1770.1,
       1985.4, 2124.5, 2471.4, 1732.4,  824.7, 2310.9, 1340.1, 1410.7,
       2385.8, 2115.4, 2264.5, 1549.9, 2425.1, 2239.8, 2044.9, 1650.1,
       1456.8, 2224.3, 1658. , 1932.4, 2270.5, 3147.1, 1493.9, 1965. ,
       2036.4, 1919.6, 1838.3, 2009.7, 2443.3, 2108.7, 2908. , 4256.6,
       1916.8, 3364.7, 1596.9, 1834.2, 1543.6, 2215. , 1971.7, 2164.9,
       3146.3, 3075.9, 2461.2,  815.6, 3726.3, 2180.8, 2496.3, 2028.1,
       2404. , 1779. , 1798.3, 3950.3, 3520.9, 2463.3, 2357.8, 1502.2,
       2817.4, 1607. , 1983.3, 1199.4, 2536.9, 2748.8, 2238.4, 2263.8,
       1198.9, 2751.8, 1490.6, 1970.7, 2336.4, 1950.8, 2647.9, 2159.5,
       2136.1, 2047.5,  808.4, 1853. , 2183.7, 2276.3, 3570.4, 1272.4,
      

In [141]:
#Mean Squared Error
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_prediction)

268794.9990551182

In [142]:
#R² Score
from sklearn.metrics import r2_score
r2_score(y_test, y_prediction)

0.6709935883388942

Conclusió: La variable independient "Room Average" té gran impacte sobre la variable objetiu (dependient) ja que el R2 Score ha baixat considerablement en el Model Random Forest Regressor en comparació amb el resultat obtingut quan hem fet servir aquesta variable.

Exercici 4 Compara el seu rendiment emprant l’aproximació traint/test o emprant totes les dades (validació interna)

- Cross Validation

In [171]:
# No he pogut fer la Cross Validation pel model Random Forest Regression

from sklearn.ensemble import RandomForestRegressor

rf = randomForest(y= df['Median Value Homes'], x= df.drop('Median Value Homes', axis = 1))
( rf.crossValidation(df.mdl, df['Median Value Homes'], p=0.10, n=99, ntree=501) )



NameError: name 'randomForest' is not defined

- Exercici 1
Crea almenys dos models de regressió diferents per intentar predir el millor possible el preu de les vivendes (MEDV) de l'arxiu adjunt.


- Regressió Lineal

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as scipy
from scipy import stats
from scipy.stats import ttest_ind
%matplotlib inline

In [34]:
df_linear_regression = pd.read_excel('Housing data.xslx.xlsx')
df_linear_regression.columns = ['Crime Rate', 'Residential Zone', 'Non Retail bsn', 'Dummy Var', 'Nitric Ox Concetration', 'Room Average', 'Owners', 'Dist Boston Jobs', 'Index Highways Distance', 'Tax rate', 'School Ratio', 'Blacks Average', 'Perc Status Popul', 'Median Value Homes']
df_linear_regression

,Crime Rate,Residential Zone,Non Retail bsn,Dummy Var,Nitric Ox Concetration,Room Average,Owners,Dist Boston Jobs,Index Highways Distance,Tax rate,School Ratio,Blacks Average,Perc Status Popul,Median Value Homes
0,632,1800,2310,0,5380,65750,6520,40900,1,2960,1530,39690,498,2400
1,2731,0,7070,0,4690,64210,7890,49671,2,2420,1780,39690,914,2160
2,2729,0,7070,0,4690,71850,6110,49671,2,2420,1780,39283,403,3470
3,3237,0,2180,0,4580,69980,4580,60622,3,2220,1870,39463,294,3340
4,6905,0,2180,0,4580,71470,5420,60622,3,2220,1870,39690,533,3620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,6263,0,11930,0,5730,65930,6910,24786,1,2730,2100,39199,967,2240
502,4527,0,11930,0,5730,61200,7670,22875,1,2730,2100,39690,908,2060
503,6076,0,11930,0,5730,69760,9100,21675,1,2730,2100,39690,564,2390
504,10959,0,11930,0,5730,67940,8930,23889,1,2730,2100,39345,648,2200


In [49]:
# Use numpy to convert to arrays
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split 

# All the features except the Target or Label Column which is 'Median Value Homes' 
Features_Variables_Independientes_X = df_linear_regression.drop('Median Value Homes', axis = 1)

# Labels are the value we want to predict:'Median Value Homes'
Labels_Variable_Dependiente_y = df_linear_regression[['Median Value Homes']]


X_train, X_test, y_train, y_test = train_test_split(Features_Variables_Independientes_X,Labels_Variable_Dependiente_y, 
 random_state=0)

In [50]:
print('Training Features Shape / Variables_Independientes_X:', X_train.shape)
print('Training Labels Shape / Variable_Dependiente_y :', y_train.shape)
print('Testing Features Shape / Variables_Independientes_X:', X_test.shape)
print('Testing Labels Shape / Variable_Dependiente_y:', y_test.shape)

Training Features Shape / Variables_Independientes_X: (379, 13)
Training Labels Shape / Variable_Dependiente_y : (379, 1)
Testing Features Shape / Variables_Independientes_X: (127, 13)
Testing Labels Shape / Variable_Dependiente_y: (127, 1)


In [51]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression(normalize=True)
#lr.fit(X, y)

In [52]:
#@AcilFarhat the number of samples in X_train and Y_train must be equal. 
#That is, X_train.shape[0] == Y_train.shape[0]. 
#Make sure you have exactly the same number of data points in X_train and Y_train. 

In [53]:
# Create Linear Model and train it 
lr.fit(X_train, y_train)



C:\Users\flore\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(


LinearRegression(normalize=True)

In [57]:
 y_pred = lr.predict(X_test)

In [58]:
y_pred

array([[2495.23328333],
       [2361.6997238 ],
       [2920.58855326],
       [1196.07051524],
       [2133.36204234],
       [1946.95489494],
       [2042.22842067],
       [2152.04405801],
       [1898.95410061],
       [1995.09829969],
       [ 492.46824392],
       [1609.6940582 ],
       [1693.59957402],
       [ 533.50840157],
       [3984.43439804],
       [3233.5498431 ],
       [2232.77257217],
       [3654.01781901],
       [3103.30061068],
       [2332.17250268],
       [2492.08649844],
       [2426.10647422],
       [2071.50442216],
       [3045.07255157],
       [2245.00923378],
       [ 987.47000639],
       [1770.32441229],
       [1797.47749971],
       [3569.93201167],
       [2079.40972009],
       [1810.55417411],
       [1768.31786503],
       [1971.35471318],
       [2379.69387321],
       [2906.52895801],
       [1923.73828377],
       [1097.81587761],
       [2456.19997843],
       [1732.91305203],
       [1520.34081708],
       [2609.33745774],
       [2087.706

- Exercici 2 Compara’ls en base al MSE i al R2

In [61]:
#Mean Squared Error
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred)

297822.45092302357

In [62]:
#R² Score
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.6354638433202129

- Exercici 3
Entrena’ls utilitzant els diferents paràmetres que admeten per intentar millorar-ne la predicció.


Per veure el impacte que poden tenir altres variables, he seleccionat aquestes variables independients: 'Owners','Perc Status Popul', 'Crime Rate', 'Nitric Ox Concetration', 'Dist Boston Jobs'

In [97]:
df_linear_regression

,Crime Rate,Residential Zone,Non Retail bsn,Dummy Var,Nitric Ox Concetration,Room Average,Owners,Dist Boston Jobs,Index Highways Distance,Tax rate,School Ratio,Blacks Average,Perc Status Popul,Median Value Homes
0,632,1800,2310,0,5380,65750,6520,40900,1,2960,1530,39690,498,2400
1,2731,0,7070,0,4690,64210,7890,49671,2,2420,1780,39690,914,2160
2,2729,0,7070,0,4690,71850,6110,49671,2,2420,1780,39283,403,3470
3,3237,0,2180,0,4580,69980,4580,60622,3,2220,1870,39463,294,3340
4,6905,0,2180,0,4580,71470,5420,60622,3,2220,1870,39690,533,3620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,6263,0,11930,0,5730,65930,6910,24786,1,2730,2100,39199,967,2240
502,4527,0,11930,0,5730,61200,7670,22875,1,2730,2100,39690,908,2060
503,6076,0,11930,0,5730,69760,9100,21675,1,2730,2100,39690,564,2390
504,10959,0,11930,0,5730,67940,8930,23889,1,2730,2100,39345,648,2200


In [101]:
# Use numpy to convert to arrays
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split 

# All the features except the Target or Label Column which is 'Median Value Homes' 
Features_Variables_Independientes_X = df_linear_regression[['Owners','Perc Status Popul', 'Crime Rate', 'Nitric Ox Concetration', 'Dist Boston Jobs']]

# Labels are the value we want to predict:'Median Value Homes'
Labels_Variable_Dependiente_y = df_linear_regression[['Median Value Homes']]


X_train, X_test, y_train, y_test = train_test_split(Features_Variables_Independientes_X,Labels_Variable_Dependiente_y, 
 random_state=0)

In [103]:
print('Training Features Shape / Variables_Independientes_X:', X_train.shape)
print('Training Labels Shape / Variable_Dependiente_y :', y_train.shape)
print('Testing Features Shape / Variables_Independientes_X:', X_test.shape)
print('Testing Labels Shape / Variable_Dependiente_y:', y_test.shape)

Training Features Shape / Variables_Independientes_X: (379, 5)
Training Labels Shape / Variable_Dependiente_y : (379, 1)
Testing Features Shape / Variables_Independientes_X: (127, 5)
Testing Labels Shape / Variable_Dependiente_y: (127, 1)


In [104]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression(normalize=True)

In [105]:
lr.fit(X_train, y_train)

C:\Users\flore\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(


LinearRegression(normalize=True)

In [106]:
 y_pred = lr.predict(X_test)

In [107]:
y_pred

array([[2757.5231127 ],
       [2721.01756723],
       [2553.3185632 ],
       [1419.90587781],
       [2334.040673  ],
       [2617.03117656],
       [2394.11618413],
       [2537.36851623],
       [2440.26571482],
       [2668.53791848],
       [ 884.27195662],
       [1548.99046586],
       [1668.38648798],
       [ 619.74986263],
       [3454.78982364],
       [2763.44456246],
       [2495.46894896],
       [2888.14912087],
       [3144.66383464],
       [2724.85104514],
       [2806.94609078],
       [2426.77694214],
       [2410.81628502],
       [2929.86003372],
       [2511.48694629],
       [2268.55774124],
       [2158.54481618],
       [1761.06314348],
       [3303.85217498],
       [2023.90491872],
       [1785.46579347],
       [1920.34956516],
       [2161.96349568],
       [2171.05233734],
       [2847.67110655],
       [1969.12718788],
       [ 754.65426704],
       [2742.3092071 ],
       [1900.25116074],
       [1275.01406253],
       [2634.9854167 ],
       [2129.935

- Exercici 2 Compara’ls en base al MSE i al R2

In [108]:
#Mean Squared Error
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred)

420633.96280233574

In [109]:
#R² Score
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.4851419438201322

- Exercici 5 No facis servir la variable del nombre d'habitacions (RM) a l’hora de fer prediccions.

In [144]:
df

,Crime Rate,Residential Zone,Non Retail bsn,Dummy Var,Nitric Ox Concetration,Room Average,Owners,Dist Boston Jobs,Index Highways Distance,Tax rate,School Ratio,Blacks Average,Perc Status Popul,Median Value Homes
0,632,1800,2310,0,5380,65750,6520,40900,1,2960,1530,39690,498,2400
1,2731,0,7070,0,4690,64210,7890,49671,2,2420,1780,39690,914,2160
2,2729,0,7070,0,4690,71850,6110,49671,2,2420,1780,39283,403,3470
3,3237,0,2180,0,4580,69980,4580,60622,3,2220,1870,39463,294,3340
4,6905,0,2180,0,4580,71470,5420,60622,3,2220,1870,39690,533,3620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,6263,0,11930,0,5730,65930,6910,24786,1,2730,2100,39199,967,2240
502,4527,0,11930,0,5730,61200,7670,22875,1,2730,2100,39690,908,2060
503,6076,0,11930,0,5730,69760,9100,21675,1,2730,2100,39690,564,2390
504,10959,0,11930,0,5730,67940,8930,23889,1,2730,2100,39345,648,2200


In [145]:
# Use numpy to convert to arrays
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split 

# All the features except the Target or Label Column which is 'Median Value Homes' 
Features_Variables_Independientes_X = df.drop(columns=['Median Value Homes','Room Average'])

# Labels are the value we want to predict:'Median Value Homes'
Labels_Variable_Dependiente_y = df[['Median Value Homes']]


X_train, X_test, y_train, y_test = train_test_split(Features_Variables_Independientes_X,Labels_Variable_Dependiente_y, 
 random_state=0)

In [146]:
print('Training Features Shape / Variables_Independientes_X:', X_train.shape)
print('Training Labels Shape / Variable_Dependiente_y :', y_train.shape)
print('Testing Features Shape / Variables_Independientes_X:', X_test.shape)
print('Testing Labels Shape / Variable_Dependiente_y:', y_test.shape)

Training Features Shape / Variables_Independientes_X: (379, 12)
Training Labels Shape / Variable_Dependiente_y : (379, 1)
Testing Features Shape / Variables_Independientes_X: (127, 12)
Testing Labels Shape / Variable_Dependiente_y: (127, 1)


In [147]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression(normalize=True)

In [148]:
lr.fit(X_train, y_train)

C:\Users\flore\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(


LinearRegression(normalize=True)

In [149]:
 y_pred = lr.predict(X_test)
    

In [150]:
y_pred

array([[2464.27425143],
       [2577.460354  ],
       [3014.91486924],
       [1444.60910997],
       [2048.37072092],
       [2092.35702398],
       [2169.90840222],
       [2283.4550458 ],
       [2329.95192281],
       [2074.00912499],
       [ 643.76713968],
       [1463.80305911],
       [1598.73760406],
       [ 847.44420378],
       [3761.24834235],
       [3146.39967007],
       [2283.14037599],
       [3593.4755072 ],
       [3159.84098654],
       [2514.52021604],
       [2488.82150133],
       [2647.44582764],
       [2100.57038207],
       [3127.81697644],
       [2428.89328162],
       [2053.03221105],
       [1821.99437109],
       [1954.3851821 ],
       [3144.76325443],
       [2087.87655555],
       [1792.76729129],
       [1789.48654919],
       [2022.82027326],
       [2502.08293177],
       [3047.04222077],
       [2106.24776016],
       [ 945.09316438],
       [2815.83313855],
       [1731.98054806],
       [1388.81595175],
       [2616.09452037],
       [2133.744

In [151]:
#Mean Squared Error
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred)

339245.10921271733

In [152]:
#R² Score
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.5847623041797405

Conclusió: En aquest model, aiximateix, la variable independient "Room Average" té gran impacte sobre la variable objetiu (dependient) ja que el R2 Score ha baixat considerablement en el Model Linear Regression en comparació amb el resultat obtingut quan hem fet servir aquesta variable.

- Exercici 4
Compara el seu rendiment emprant l’aproximació traint/test o emprant totes les dades (validació interna).


- Cross Validation 

In [158]:
from sklearn.model_selection import cross_val_score
print(cross_val_score(lr, Features_Variables_Independientes_X, Labels_Variable_Dependiente_y, cv=2))

[ 0.56561432 -6.72587239]


C:\Users\flore\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(
C:\Users\flore\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model =

In [159]:
from sklearn.model_selection import cross_val_score
print(cross_val_score(lr, Features_Variables_Independientes_X, Labels_Variable_Dependiente_y, cv=5))

[0.46450601 0.61522064 0.43616681 0.57879898 0.37296639]


C:\Users\flore\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(
C:\Users\flore\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model =

In [26]:
print(cross_val_score(bosque,[['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT']],
                      #variable objetivo
                     Housing['MEDV'],
                      #pliegues
                     cv=5))

ValueError: Found input variables with inconsistent numbers of samples: [1, 506]

- Exercici 5
No facis servir la variable del nombre d'habitacions (RM) a l’hora de fer prediccions.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as scipy
from scipy import stats
from scipy.stats import ttest_ind
%matplotlib inline
df = pd.read_excel('Housing data.xslx.xlsx')
df

In [ ]:
df.columns = ['Crime Rate', 'Residential Zone', 'Non Retail bsn', 'Dummy Var', 'Nitric Ox Concetration', 'Room Average', 'Owners', 'Dist Boston Jobs', 'Index Highways Distance', 'Tax rate', 'School Ratio', 'Blacks Average', 'Perc Status Popul', 'Median Value Homes']
df

In [ ]:
df.columns


In [ ]:
# Labels are the values we want to predict
Lo_que_queremos_averiguar = df['Median Value Homes']
# Remove the labels from the features
# Sacamos la columna 'Room Average' y 'Median Value Homes'
La_informacion_que_tenemos = df[['Crime Rate', 'Residential Zone', 'Non Retail bsn', 'Dummy Var',
       'Nitric Ox Concetration', 'Owners', 'Dist Boston Jobs',
       'Index Highways Distance', 'Tax rate', 'School Ratio', 'Blacks Average',
       'Perc Status Popul']]
# Saving feature names for later use
feature_list = list(df.columns)

In [ ]:
from sklearn.model_selection import train_test_split 
train_features, test_features, train_labels, test_labels = train_test_split(La_informacion_que_tenemos,Lo_que_queremos_averiguar , test_size = 0.25, random_state = 42)



In [ ]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

In [ ]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(train_features, train_labels);


In [ ]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)
# Calculate the absolute errors
errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

In [ ]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')